In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
fastballDL = pd.read_csv('DraftLeagueFastball.csv')
fastballMLB = pd.read_csv('MLBFastball.csv')

In [3]:
fastballDL['league'] = 'Draft League'
fastballMLB['league'] = 'MLB'

In [4]:
fastballDL = fastballDL.astype(str)
fastballMLB = fastballMLB.astype(str)

In [5]:
pitchersDL = list(set(fastballDL['pitcher']))

In [6]:
fastball = pd.concat([fastballDL, fastballMLB])

In [7]:
def combined_features(row):
    return row['avgvelo']+" "+row['avgspinrate']+" "+row['avghorzbreak']+" "+row['avgvertbreak']+" "+row['avgeffectvelo']

In [8]:
#fig, ax = plt.subplots(figsize=(100,100))
#sns.heatmap(cosine_sim,cmap='magma',ax=ax)

In [9]:
def get_index_from_pitcher(pitcher):
    return fastball[fastball.pitcher == pitcher].index[0]

In [10]:
def get_pitcher_from_index(index):
    return fastball[fastball.index == index].values[0][0]

In [11]:
mlb_draft_pitcher = []
mlb_comp = []
cos_value = []

for x in pitchersDL:
    
    mlb_draft_pitcher.append(x)
    
    try:

        fastball = pd.concat([fastballDL, fastballMLB])

        fastball["combined_features"] = fastball.apply(combined_features, axis =1)

        cv = CountVectorizer()
        count_matrix = cv.fit_transform(fastball["combined_features"])

        cosine_sim = cosine_similarity(count_matrix)

        pitcher_like = x
        fastball = fastball[(fastball['pitcher'] == pitcher_like) | (fastball['league'] == 'MLB')]

        player_index = get_index_from_pitcher(pitcher_like)

        similar_pitchers = list(enumerate(cosine_sim[player_index]))

        sorted_similar_pitchers = sorted(similar_pitchers, key=lambda x:x[1], reverse=True)

        mlb_comp.append(get_pitcher_from_index(sorted_similar_pitchers[1][0]))
        
        cos_value.append(sorted_similar_pitchers[1][1])
                
    except IndexError:
        mlb_comp.append('No Comp')
        cos_value.append('No Similarity')
        pass

In [12]:
cosine_comp = pd.DataFrame(list(zip(mlb_draft_pitcher, mlb_comp, cos_value)), columns=['MLB DL Pitcher', 'MLB Comp', "Cosine Value"])
cosine_comp

,MLB DL Pitcher,MLB Comp,Cosine Value
0,"Fruit, Zach","Bostick, Akeem",0.353553
1,"Morris, Gunner","Ynoa, Huascar",0.333333
2,"Alba, Miguel","Pérez, Cionel",0.286039
3,"Johnson, Tyler","Sánchez, Sixto",0.3
4,"Hansel, Luke",No Comp,No Similarity
...,...,...,...
142,"Bortka, Josh","Burdi, Nick",0.365148
143,"Peaden, Jacob","Yamamoto, Jordan",0.421637
144,"Cox, Brennan","Green, Chad",0.316228
145,"Rodriguez, Mauricio","Grace, Matt",0.46291


In [13]:
cosine_comp.to_csv('fastball_cosine_comp.csv', sep='\t')